In [1]:
import os

In [2]:
%pwd

'e:\\Machine Learning\\MLFlow_Project_by_Krish_Naik\\Kidney-Disease-Classification-Deep-Learning-Project\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'e:\\Machine Learning\\MLFlow_Project_by_Krish_Naik\\Kidney-Disease-Classification-Deep-Learning-Project'

In [23]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [24]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [25]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=CONFIG_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_url = config.source_url,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )

        return data_ingestion_config

In [26]:
import os
import gdown
import zipfile
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [29]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self) -> str:
        """
        Fetch data from url
        """
        try:
            dataset_url = self.config.source_url
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split('/')[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id, zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")
            
        except Exception as e:
            raise e
        
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """

        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [30]:
# Building the pipeling

try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()

except Exception as e:
    raise e

[2025-09-01 08:54:11,461: INFO: common: yaml file: <_io.TextIOWrapper name='config\\config.yaml' mode='r' encoding='cp1252'> loaded successfully]
[2025-09-01 08:54:11,465: INFO: common: yaml file: <_io.TextIOWrapper name='config\\config.yaml' mode='r' encoding='cp1252'> loaded successfully]
[2025-09-01 08:54:11,468: INFO: common: Created directory at: artifacts]
[2025-09-01 08:54:11,469: INFO: common: Created directory at: artifacts/data_ingestion]
[2025-09-01 08:54:11,471: INFO: 4240413249: Downloading data from https://drive.google.com/file/d/1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3&confirm=t&uuid=a934fb97-47b4-4166-8ae1-4687ec7b0c52
To: e:\Machine Learning\MLFlow_Project_by_Krish_Naik\Kidney-Disease-Classification-Deep-Learning-Project\artifacts\data_ingestion\data.zip
100%|██████████| 57.7M/57.7M [00:09<00:00, 6.18MB/s]


[2025-09-01 08:54:32,969: INFO: 4240413249: Downloaded data from https://drive.google.com/file/d/1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3/view?usp=sharing into file artifacts/data_ingestion/data.zip]
